In [1]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

In [2]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler

In [4]:
spark = SparkSession.builder.appName("Clustering Using SparkML").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/01 15:54:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
df = spark.read.csv("customers.csv", header=True, inferSchema=True)
df.show(5)

+----------+----+-------+-----------+
|Fresh_Food|Milk|Grocery|Frozen_Food|
+----------+----+-------+-----------+
|     12669|9656|   7561|        214|
|      7057|9810|   9568|       1762|
|      6353|8808|   7684|       2405|
|     13265|1196|   4221|       6404|
|     22615|5410|   7198|       3915|
+----------+----+-------+-----------+
only showing top 5 rows



In [6]:
df.printSchema()

root
 |-- Fresh_Food: integer (nullable = true)
 |-- Milk: integer (nullable = true)
 |-- Grocery: integer (nullable = true)
 |-- Frozen_Food: integer (nullable = true)



In [7]:
df.show(n=5, truncate=False)

+----------+----+-------+-----------+
|Fresh_Food|Milk|Grocery|Frozen_Food|
+----------+----+-------+-----------+
|12669     |9656|7561   |214        |
|7057      |9810|9568   |1762       |
|6353      |8808|7684   |2405       |
|13265     |1196|4221   |6404       |
|22615     |5410|7198   |3915       |
+----------+----+-------+-----------+
only showing top 5 rows



In [9]:
feature_cols = df.columns
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
df_transformed = assembler.transform(df)

In [11]:
kmeans = KMeans(k=3)

In [12]:
model = kmeans.fit(df_transformed)

23/12/01 16:18:52 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


In [13]:
predictions = model.transform(df_transformed)

In [14]:
predictions.show(5)

+----------+----+-------+-----------+--------------------+----------+
|Fresh_Food|Milk|Grocery|Frozen_Food|            features|prediction|
+----------+----+-------+-----------+--------------------+----------+
|     12669|9656|   7561|        214|[12669.0,9656.0,7...|         1|
|      7057|9810|   9568|       1762|[7057.0,9810.0,95...|         1|
|      6353|8808|   7684|       2405|[6353.0,8808.0,76...|         1|
|     13265|1196|   4221|       6404|[13265.0,1196.0,4...|         1|
|     22615|5410|   7198|       3915|[22615.0,5410.0,7...|         0|
+----------+----+-------+-----------+--------------------+----------+
only showing top 5 rows



In [15]:
predictions.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  340|
|         2|   25|
|         0|   75|
+----------+-----+



In [16]:
spark.stop()